In [3]:
# Importar librerías necesarias - Config. InfluxDB y Clickhouse
import pandas as pd
import numpy as np
import os
import sys
import logging
from datetime import datetime
import clickhouse_connect
from influxdb_client import InfluxDBClient
import gc

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Configuración de InfluxDB
INFLUX_CONFIG = {
    'url': "http://146.83.153.212:27017", #"http://172.24.61.95:27017"
    'token': "piDbFR_bfRWO5Epu1IS96WbkNpSZZCYgwZZR29PcwUsxXwKdIyLMhVAhU4-5ohWeXIsX7Dp_X-WiPIDx0beafg==",
    'org': "atamostec",
    'timeout': 300000
}

# Configuración de Clickhouse
CLICKHOUSE_CONFIG = {
    'host': "146.83.153.212", #"172.24.61.95"
    'port': "30091",
    'user': "default",
    'password': "Psda2020"
}

# Configuración de fechas
START_DATE = pd.to_datetime('01/07/2024', dayfirst=True).tz_localize('UTC')
END_DATE = pd.to_datetime('31/12/2025', dayfirst=True).tz_localize('UTC')

# Directorio de salida - ruta absoluta desde el notebook en download/
OUTPUT_DIR = "/home/nicole/SR/SOILING/datos"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Mostrar configuración
logger.info(f"Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"Directorio de salida: {OUTPUT_DIR}")

2025-08-07 12:12:12 - INFO - Rango de fechas: 2024-07-01 a 2025-12-31
2025-08-07 12:12:12 - INFO - Directorio de salida: /home/nicole/SR/SOILING/datos


In [4]:
class InfluxDBManager:
    def __init__(self, config):
        self.config = config
        self.client = None
        self.query_api = None
        
    def connect(self):
        try:
            self.client = InfluxDBClient(
                url=self.config['url'],
                token=self.config['token'],
                org=self.config['org'],
                timeout=self.config['timeout']
            )
            self.query_api = self.client.query_api()
            logger.info("Cliente InfluxDB y query_api inicializados.")
            return True
        except Exception as e:
            logger.error(f"Error al conectar con InfluxDB: {e}")
            return False
            
    def disconnect(self):
        if self.client:
            self.client.close()
            logger.info("Conexión a InfluxDB cerrada.")
            
    def query_influxdb(self, bucket, tables, attributes, start_date, stop_date):
        try:
            # Convertir fechas al formato correcto para InfluxDB
            start_str = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            stop_str = stop_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            
            # Construir la lista de atributos en formato correcto
            attributes_str = " or ".join([f'r["_field"] == "{attr}"' for attr in attributes])

            query = f'''
            from(bucket: "{bucket}")
                |> range(start: {start_str}, stop: {stop_str})
                |> filter(fn: (r) => {" or ".join([f'r["_measurement"] == "{table}"' for table in tables])})
                |> filter(fn: (r) => {attributes_str})
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
            '''
            
            logger.info(f"Consultando InfluxDB: bucket={bucket}, tables={tables}, attributes={attributes}")
            
            result = self.query_api.query_data_frame(query)
            
            if result.empty:
                logger.warning("No se encontraron datos en la consulta.")
                return None
                
            return result
            
        except Exception as e:
            logger.error(f"Error en la consulta a InfluxDB: {e}")
            return None

In [5]:
# 🔋 DESCARGA IV600 
def download_iv600(start_date, end_date, output_dir):
    """Descarga y procesa datos de IV600 desde Clickhouse."""
    logger.info("Iniciando descarga de datos IV600...")
    client = None
    
    try:
        # Conectar a Clickhouse
        logger.info("Conectando a Clickhouse...")
        client = clickhouse_connect.get_client(
            host=CLICKHOUSE_CONFIG['host'],
            port=CLICKHOUSE_CONFIG['port'],
            username=CLICKHOUSE_CONFIG['user'],
            password=CLICKHOUSE_CONFIG['password']
        )
        logger.info("Conexión a Clickhouse establecida")
        
        # Consultar datos
        logger.info("Consultando datos IV600...")
        query = "SELECT * FROM ref_data.iv_curves_trazador_manual"
        data_iv_curves = client.query(query)
        logger.info(f"Datos obtenidos: {len(data_iv_curves.result_set)} registros")
        
        # Procesar datos
        logger.info("Procesando datos...")
        curves_list = []
        for curve in data_iv_curves.result_set:
            currents = curve[4]
            voltages = curve[3]
            powers = [currents[i] * voltages[i] for i in range(len(currents))]
            timestamp = curve[0]
            module = curve[2]
            pmp = max(powers)
            isc = max(currents)
            voc = max(voltages)
            imp = currents[np.argmax(powers)]
            vmp = voltages[np.argmax(powers)]
            curves_list.append([timestamp, module, pmp, isc, voc, imp, vmp])

        # Crear DataFrame
        logger.info("Creando DataFrame...")
        column_names = ["timestamp", "module", "pmp", "isc", "voc", "imp", "vmp"]
        df_curves = pd.DataFrame(curves_list, columns=column_names)
        
        # Convertir timestamp a datetime y asegurar que esté en UTC
        df_curves['timestamp'] = pd.to_datetime(df_curves['timestamp'])
        if df_curves['timestamp'].dt.tz is None:
            df_curves['timestamp'] = df_curves['timestamp'].dt.tz_localize('UTC')
        else:
            df_curves['timestamp'] = df_curves['timestamp'].dt.tz_convert('UTC')
        
        df_curves.set_index('timestamp', inplace=True)
        
        # Mostrar información sobre el rango de fechas en los datos
        logger.info(f"Rango de fechas en los datos:")
        logger.info(f"Fecha más antigua: {df_curves.index.min()}")
        logger.info(f"Fecha más reciente: {df_curves.index.max()}")
        
        # Filtrar por fecha usando query para mayor flexibilidad
        logger.info(f"Filtrando datos entre {start_date} y {end_date}...")
        df_curves = df_curves.query('@start_date <= index <= @end_date')
        
        if len(df_curves) == 0:
            logger.warning("No se encontraron datos en el rango de fechas especificado.")
            logger.info("Ajustando el rango de fechas al rango disponible en los datos...")
            df_curves = df_curves.sort_index()
        else:
            logger.info(f"Se encontraron {len(df_curves)} registros en el rango especificado.")

        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_iv600_data.csv')
        logger.info(f"Guardando datos en: {output_filepath}")
        df_curves.to_csv(output_filepath)
        logger.info(f"Datos guardados exitosamente. Total de registros: {len(df_curves)}")
        logger.info(f"Rango de fechas: {df_curves.index.min()} a {df_curves.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos IV600: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False
    finally:
        if client:
            logger.info("Cerrando conexión a Clickhouse...")
            client.close()
            logger.info("Conexión a Clickhouse cerrada")
            

In [6]:
# 🔋 DESCARGA PV GLASSES
def download_pv_glasses(influx_client, start_date, end_date, output_dir):
    """Descarga y procesa datos de PV Glasses."""
    logger.info("Iniciando descarga de datos PV Glasses...")
    
    try:
        # Configuración de la consulta
        bucket = "meteo_psda"
        tables = ["6852_Ftc"]
        attributes = ["R_FC1_Avg", "R_FC2_Avg", "R_FC3_Avg", "R_FC4_Avg", "R_FC5_Avg"]
        
        # Obtener datos
        df_glasses = influx_client.query_influxdb(bucket, tables, attributes, start_date, end_date)
        
        if df_glasses is None or df_glasses.empty:
            logger.warning("No se obtuvieron datos de PV Glasses")
            return False
            
        # Asegurar que el índice sea DatetimeIndex
        if '_time' in df_glasses.columns:
            df_glasses.set_index('timestamp', inplace=True)
        elif 'time' in df_glasses.columns:
            df_glasses.set_index('time', inplace=True)
            
        # Convertir el índice a DatetimeIndex si no lo es
        if not isinstance(df_glasses.index, pd.DatetimeIndex):
            df_glasses.index = pd.to_datetime(df_glasses.index)
            
        # Filtrar por horario (13:00 a 18:00)
        df_glasses = df_glasses.between_time('13:00', '18:00')
        
        # Seleccionar solo las columnas numéricas para el cálculo
        numeric_columns = df_glasses.select_dtypes(include=[np.number]).columns
        df_glasses_numeric = df_glasses[numeric_columns]
        
        # Calcular referencia (promedio de R_FC1_Avg)
        if 'R_FC1_Avg' in df_glasses_numeric.columns:
            df_glasses['Ref'] = df_glasses_numeric['R_FC1_Avg'].mean()
        
        # Calcular datos diarios solo para columnas numéricas
        df_glasses_daily = df_glasses_numeric.resample('1d').sum().div(60000)
        
        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_pv_glasses_data.csv')
        daily_output_filepath = os.path.join(output_dir, 'raw_pv_glasses_daily_data.csv')
        
        df_glasses.to_csv(output_filepath)
        df_glasses_daily.to_csv(daily_output_filepath)
        
        logger.info(f"Datos PV Glasses guardados exitosamente")
        logger.info(f"Total de registros: {len(df_glasses)}")
        logger.info(f"Rango de fechas: {df_glasses.index.min()} a {df_glasses.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos PV Glasses: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False

In [7]:
# 🔋 DESCARGA DUSTIQ DESDE INFLUXDB
def download_dustiq(influx_client, start_date, end_date, output_dir):
    """Descarga y procesa datos de DustIQ."""
    logger.info("Iniciando descarga de datos DustIQ...")
    
    try:
        # Configuración de la consulta
        bucket = "PSDA"
        tables = ["DustIQ"]
        attributes = ["SR_C11_Avg", "SR_C12_Avg"]
        
        # Obtener datos
        df_dustiq = influx_client.query_influxdb(bucket, tables, attributes, start_date, end_date)
        
        if df_dustiq is None or df_dustiq.empty:
            logger.warning("No se obtuvieron datos de DustIQ")
            return False
            
        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_dustiq_data.csv')
        df_dustiq.to_csv(output_filepath)
        
        logger.info(f"Datos DustIQ guardados exitosamente")
        logger.info(f"Total de registros: {len(df_dustiq)}")
        logger.info(f"Rango de fechas: {df_dustiq.index.min()} a {df_dustiq.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos DustIQ: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False

In [8]:
# 🔋 DESCARGA DUSTIQ DESDE CLICKHOUSE
def download_dustiq_clickhouse(start_date, end_date, output_dir):
    """Descarga y procesa datos de DustIQ desde ClickHouse."""
    logger.info("Iniciando descarga de datos DustIQ desde ClickHouse...")
    client = None
    
    try:
        # Conectar a Clickhouse
        logger.info("Conectando a Clickhouse...")
        client = clickhouse_connect.get_client(
            host=CLICKHOUSE_CONFIG['host'],
            port=CLICKHOUSE_CONFIG['port'],
            username=CLICKHOUSE_CONFIG['user'],
            password=CLICKHOUSE_CONFIG['password']
        )
        logger.info("Conexión a Clickhouse establecida")
        
        # Convertir fechas al formato correcto para ClickHouse
        start_str = start_date.strftime("%Y-%m-%d %H:%M:%S")
        end_str = end_date.strftime("%Y-%m-%d %H:%M:%S")
        
        # Consultar datos de dustiq desde el bucket PSDA
        logger.info("Consultando datos DustIQ desde ClickHouse...")
        query = f"""
        SELECT 
            Stamptime,
            Attribute,
            Measure
        FROM PSDA.dustiq 
        WHERE Stamptime >= '{start_str}' AND Stamptime <= '{end_str}'
        AND Attribute IN ('SR_C11_Avg', 'SR_C12_Avg')
        ORDER BY Stamptime, Attribute
        """
        
        logger.info(f"Ejecutando consulta: {query[:100]}...")
        result = client.query(query)
        
        if not result.result_set:
            logger.warning("No se encontraron datos de DustIQ en ClickHouse")
            return False
            
        logger.info(f"Datos obtenidos: {len(result.result_set)} registros")
        
        # Convertir a DataFrame
        logger.info("Procesando datos...")
        df_dustiq = pd.DataFrame(result.result_set, columns=['Stamptime', 'Attribute', 'Measure'])        
        # Convertir Stamptime a datetime y asegurar que esté en UTC
        df_dustiq['Stamptime'] = pd.to_datetime(df_dustiq['Stamptime'])
        if df_dustiq['Stamptime'].dt.tz is None:
            df_dustiq['Stamptime'] = df_dustiq['Stamptime'].dt.tz_localize('UTC')
        else:
            df_dustiq['Stamptime'] = df_dustiq['Stamptime'].dt.tz_convert('UTC')

        # Pivotar los datos para convertir de long format a wide format
        logger.info("Pivotando datos de long format a wide format...")

        # Primero, manejar duplicados agregando por promedio
        logger.info("Manejando duplicados agrupando por promedio...")
        df_dustiq_grouped = df_dustiq.groupby(['Stamptime', 'Attribute'])['Measure'].mean().reset_index()

        # Ahora hacer el pivot sin duplicados
        df_dustiq_pivot = df_dustiq_grouped.pivot(index='Stamptime', columns='Attribute', values='Measure')

        # Renombrar el índice
        df_dustiq_pivot.index.name = 'timestamp'
        # Mostrar información sobre el rango de fechas en los datos
        logger.info(f"Rango de fechas en los datos:")
        logger.info(f"Fecha más antigua: {df_dustiq_pivot.index.min()}")
        logger.info(f"Fecha más reciente: {df_dustiq_pivot.index.max()}")

        # Verificar que hay datos en el rango especificado
        if len(df_dustiq_pivot) == 0:
            logger.warning("No se encontraron datos en el rango de fechas especificado.")
            return False

        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_dustiq_data.csv')
        logger.info(f"Guardando datos en: {output_filepath}")
        df_dustiq_pivot.to_csv(output_filepath)

        logger.info(f"Datos DustIQ desde ClickHouse guardados exitosamente")
        logger.info(f"Total de registros: {len(df_dustiq_pivot)}")
        logger.info(f"Rango de fechas: {df_dustiq_pivot.index.min()} a {df_dustiq_pivot.index.max()}")

        # Mostrar estadísticas básicas
        logger.info("Estadísticas de los datos:")
        if 'SR_C11_Avg' in df_dustiq_pivot.columns:
            logger.info(f"SR_C11_Avg - Rango: {df_dustiq_pivot['SR_C11_Avg'].min():.3f} a {df_dustiq_pivot['SR_C11_Avg'].max():.3f}")
        if 'SR_C12_Avg' in df_dustiq_pivot.columns:
            logger.info(f"SR_C12_Avg - Rango: {df_dustiq_pivot['SR_C12_Avg'].min():.3f} a {df_dustiq_pivot['SR_C12_Avg'].max():.3f}")
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos DustIQ desde ClickHouse: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False
    finally:
        if client:
            logger.info("Cerrando conexión a Clickhouse...")
            client.close()
            logger.info("Conexión a Clickhouse cerrada")

In [9]:
# 🔋 DESCARGA PVSTAND DESDE CLICKHOUSE (CON IDENTIFICACIÓN DE MÓDULO)
# ============================================================================

def download_pvstand_clickhouse(start_date, end_date, output_dir):
    """Descarga y procesa datos de PVStand desde ClickHouse."""
    logger.info("Iniciando descarga de datos PVStand desde ClickHouse...")
    client = None
    
    try:
        # Conectar a Clickhouse
        logger.info("Conectando a Clickhouse...")
        client = clickhouse_connect.get_client(
            host=CLICKHOUSE_CONFIG['host'],
            port=CLICKHOUSE_CONFIG['port'],
            username=CLICKHOUSE_CONFIG['user'],
            password=CLICKHOUSE_CONFIG['password']
        )
        logger.info("Conexión a Clickhouse establecida")
        
        # Convertir fechas al formato correcto para ClickHouse
        start_str = start_date.strftime("%Y-%m-%d %H:%M:%S")
        end_str = end_date.strftime("%Y-%m-%d %H:%M:%S")
        
        # Consultar datos de PVStand desde las tablas perc1fixed y perc2fixed
        logger.info("Consultando datos PVStand desde ClickHouse...")
        query = f"""
        SELECT 
            timestamp,
            'perc1fixed' as module,
            pmax,
            imax,
            umax
        FROM PSDA.perc1fixed 
        WHERE timestamp >= '{start_str}' AND timestamp <= '{end_str}'
        
        UNION ALL
        
        SELECT 
            timestamp,
            'perc2fixed' as module,
            pmax,
            imax,
            umax
        FROM PSDA.perc2fixed 
        WHERE timestamp >= '{start_str}' AND timestamp <= '{end_str}'
        
        ORDER BY timestamp
        """
        
        logger.info(f"Ejecutando consulta: {query[:100]}...")
        result = client.query(query)
        
        if not result.result_set:
            logger.warning("No se encontraron datos de PVStand en ClickHouse")
            return False
            
        logger.info(f"Datos obtenidos: {len(result.result_set)} registros")
        
        # Convertir a DataFrame
        logger.info("Procesando datos...")
        df_pvstand = pd.DataFrame(result.result_set, columns=['timestamp', 'module', 'pmax', 'imax', 'umax'])
        
        # Convertir timestamp a datetime y asegurar que esté en UTC
        df_pvstand['timestamp'] = pd.to_datetime(df_pvstand['timestamp'])
        if df_pvstand['timestamp'].dt.tz is None:
            df_pvstand['timestamp'] = df_pvstand['timestamp'].dt.tz_localize('UTC')
        else:
            df_pvstand['timestamp'] = df_pvstand['timestamp'].dt.tz_convert('UTC')

        # Establecer timestamp como índice
        df_pvstand.set_index('timestamp', inplace=True)
        
        # Ordenar por timestamp (importante para series temporales)
        logger.info("Ordenando datos por timestamp...")
        df_pvstand = df_pvstand.sort_index()
        
        # Mostrar información sobre el rango de fechas en los datos
        logger.info(f"Rango de fechas en los datos:")
        logger.info(f"Fecha más antigua: {df_pvstand.index.min()}")
        logger.info(f"Fecha más reciente: {df_pvstand.index.max()}")

        # Verificar que hay datos en el rango especificado
        if len(df_pvstand) == 0:
            logger.warning("No se encontraron datos en el rango de fechas especificado.")
            return False

        # Mostrar distribución por módulo
        module_counts = df_pvstand['module'].value_counts()
        logger.info("Distribución por módulo:")
        for module, count in module_counts.items():
            logger.info(f"   - {module}: {count} registros")

        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_pvstand_clickhouse_data.csv')
        logger.info(f"Guardando datos en: {output_filepath}")
        df_pvstand.to_csv(output_filepath)

        logger.info(f"Datos PVStand desde ClickHouse guardados exitosamente")
        logger.info(f"Total de registros: {len(df_pvstand)}")
        logger.info(f"Rango de fechas: {df_pvstand.index.min()} a {df_pvstand.index.max()}")

        # Mostrar estadísticas básicas por módulo
        logger.info("Estadísticas de los datos por módulo:")
        for module in ['perc1fixed', 'perc2fixed']:
            if module in df_pvstand['module'].values:
                module_data = df_pvstand[df_pvstand['module'] == module]
                logger.info(f"\n{module}:")
                logger.info(f"   pmax - Rango: {module_data['pmax'].min():.3f} a {module_data['pmax'].max():.3f}")
                logger.info(f"   imax - Rango: {module_data['imax'].min():.3f} a {module_data['imax'].max():.3f}")
                logger.info(f"   umax - Rango: {module_data['umax'].min():.3f} a {module_data['umax'].max():.3f}")
        
        # Mostrar información sobre la estructura de datos
        logger.info("\nEstructura de datos del PVStand:")
        logger.info(f"   - module: Identificador del módulo (perc1fixed/perc2fixed)")
        logger.info(f"   - pmax: Potencia máxima del módulo")
        logger.info(f"   - imax: Corriente máxima del módulo")
        logger.info(f"   - umax: Voltaje máximo del módulo")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos PVStand desde ClickHouse: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False
    finally:
        if client:
            logger.info("Cerrando conexión a Clickhouse...")
            client.close()
            logger.info("Conexión a Clickhouse cerrada")

In [10]:
# 🔋 DESCARGA PVSTAND DESDE INFLUXDB
def download_pvstand(influx_client, start_date, end_date, output_dir):
    """Descarga y procesa datos de PVStand."""
    logger.info("Iniciando descarga de datos PVStand...")
    
    try:
        # Configuración de la consulta
        bucket = "PSDA"
        tables = ["PERC1_fixed_1MD43420160719", "PERC2_fixed_1MD43920160719"]
        attributes = ["Imax", "Umax", "Pmax"]
        
        # Obtener datos
        df_pvstand = influx_client.query_influxdb(bucket, tables, attributes, start_date, end_date)
        
        if df_pvstand is None or df_pvstand.empty:
            logger.warning("No se obtuvieron datos de PVStand")
            return False
            
        # Guardar datos
        output_filepath = os.path.join(output_dir, 'raw_pvstand_iv_data.csv')
        df_pvstand.to_csv(output_filepath)
        
        logger.info(f"Datos PVStand guardados exitosamente")
        logger.info(f"Total de registros: {len(df_pvstand)}")
        logger.info(f"Rango de fechas: {df_pvstand.index.min()} a {df_pvstand.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos PVStand: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False

In [11]:
# 🔋 DESCARGA SOILING KIT DESDE INFLUXDB
def download_soiling_kit(influx_client, start_date, end_date, output_dir):
    """Descarga y procesa datos del Soiling Kit."""
    logger.info("Iniciando descarga de datos del Soiling Kit...")
    
    try:
        # Configuración de la consulta
        bucket = "PSDA"
        tables = ["soilingkit"]
        attributes = ["Isc(e)", "Isc(p)", "Te(C)", "Tp(C)"]
        
        # Obtener datos
        df_sk = influx_client.query_influxdb(bucket, tables, attributes, start_date, end_date)
        
        if df_sk is None or df_sk.empty:
            logger.warning("No se obtuvieron datos del Soiling Kit")
            return False
            
        # Guardar datos
        output_filepath = os.path.join(output_dir, 'soiling_kit_raw_data.csv')
        df_sk.to_csv(output_filepath)
        
        logger.info(f"Datos del Soiling Kit guardados exitosamente")
        logger.info(f"Total de registros: {len(df_sk)}")
        logger.info(f"Rango de fechas: {df_sk.index.min()} a {df_sk.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos del Soiling Kit: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False

In [12]:
# 🌪️ DESCARGA SOILING KIT DESDE CLICKHOUSE
# ============================================================================

def download_soiling_kit_clickhouse(start_date, end_date, output_dir):
    """Descarga y procesa datos del Soiling Kit desde ClickHouse."""
    logger.info("Iniciando descarga de datos del Soiling Kit desde ClickHouse...")
    client = None
    
    try:
        # Conectar a Clickhouse
        logger.info("Conectando a Clickhouse...")
        client = clickhouse_connect.get_client(
            host=CLICKHOUSE_CONFIG['host'],
            port=CLICKHOUSE_CONFIG['port'],
            username=CLICKHOUSE_CONFIG['user'],
            password=CLICKHOUSE_CONFIG['password']
        )
        logger.info("Conexión a Clickhouse establecida")
        
        # Convertir fechas al formato correcto para ClickHouse
        start_str = start_date.strftime("%Y-%m-%d %H:%M:%S")
        end_str = end_date.strftime("%Y-%m-%d %H:%M:%S")
        
        # Consultar datos del Soiling Kit desde PSDA.soilingkit
        logger.info("Consultando datos del Soiling Kit desde ClickHouse...")
        query = f"""
        SELECT 
            Stamptime,
            Attribute,
            Measure
        FROM PSDA.soilingkit 
        WHERE Stamptime >= '{start_str}' AND Stamptime <= '{end_str}'
        AND Attribute IN ('Isc(e)', 'Isc(p)', 'Te(C)', 'Tp(C)')
        ORDER BY Stamptime, Attribute
        """
        
        logger.info(f"Ejecutando consulta: {query[:100]}...")
        result = client.query(query)
        
        if not result.result_set:
            logger.warning("No se encontraron datos del Soiling Kit en ClickHouse")
            return False
            
        logger.info(f"Datos obtenidos: {len(result.result_set)} registros")
        
        # Convertir a DataFrame
        logger.info("Procesando datos...")
        df_soilingkit = pd.DataFrame(result.result_set, columns=['Stamptime', 'Attribute', 'Measure'])
        
        # Convertir Stamptime a datetime y asegurar que esté en UTC
        df_soilingkit['Stamptime'] = pd.to_datetime(df_soilingkit['Stamptime'])
        if df_soilingkit['Stamptime'].dt.tz is None:
            df_soilingkit['Stamptime'] = df_soilingkit['Stamptime'].dt.tz_localize('UTC')
        else:
            df_soilingkit['Stamptime'] = df_soilingkit['Stamptime'].dt.tz_convert('UTC')

        # Pivotar los datos para convertir de long format a wide format
        logger.info("Pivotando datos de long format a wide format...")

        # Primero, manejar duplicados agregando por promedio
        logger.info("Manejando duplicados agrupando por promedio...")
        df_soilingkit_grouped = df_soilingkit.groupby(['Stamptime', 'Attribute'])['Measure'].mean().reset_index()

        # Ahora hacer el pivot sin duplicados
        df_soilingkit_pivot = df_soilingkit_grouped.pivot(index='Stamptime', columns='Attribute', values='Measure')

        # Renombrar el índice
        df_soilingkit_pivot.index.name = 'timestamp'
        
        # Mostrar información sobre el rango de fechas en los datos
        logger.info(f"Rango de fechas en los datos:")
        logger.info(f"Fecha más antigua: {df_soilingkit_pivot.index.min()}")
        logger.info(f"Fecha más reciente: {df_soilingkit_pivot.index.max()}")

        # Verificar que hay datos en el rango especificado
        if len(df_soilingkit_pivot) == 0:
            logger.warning("No se encontraron datos en el rango de fechas especificado.")
            return False

        # Guardar datos
        output_filepath = os.path.join(output_dir, 'soiling_kit_raw_data.csv')
        logger.info(f"Guardando datos en: {output_filepath}")
        df_soilingkit_pivot.to_csv(output_filepath)

        logger.info(f"Datos del Soiling Kit desde ClickHouse guardados exitosamente")
        logger.info(f"Total de registros: {len(df_soilingkit_pivot)}")
        logger.info(f"Rango de fechas: {df_soilingkit_pivot.index.min()} a {df_soilingkit_pivot.index.max()}")

        # Mostrar estadísticas básicas
        logger.info("Estadísticas de los datos:")
        if 'Isc(e)' in df_soilingkit_pivot.columns:
            logger.info(f"Isc(e) - Rango: {df_soilingkit_pivot['Isc(e)'].min():.3f} a {df_soilingkit_pivot['Isc(e)'].max():.3f}")
        if 'Isc(p)' in df_soilingkit_pivot.columns:
            logger.info(f"Isc(p) - Rango: {df_soilingkit_pivot['Isc(p)'].min():.3f} a {df_soilingkit_pivot['Isc(p)'].max():.3f}")
        if 'Te(C)' in df_soilingkit_pivot.columns:
            logger.info(f"Te(C) - Rango: {df_soilingkit_pivot['Te(C)'].min():.1f} a {df_soilingkit_pivot['Te(C)'].max():.1f}")
        if 'Tp(C)' in df_soilingkit_pivot.columns:
            logger.info(f"Tp(C) - Rango: {df_soilingkit_pivot['Tp(C)'].min():.1f} a {df_soilingkit_pivot['Tp(C)'].max():.1f}")
        
        # Mostrar información sobre la estructura de datos
        logger.info("Estructura de datos del Soiling Kit:")
        logger.info(f"   - Isc(e): Corriente de cortocircuito de la celda limpia (referencia)")
        logger.info(f"   - Isc(p): Corriente de cortocircuito de la celda sucia (panel)")
        logger.info(f"   - Te(C): Temperatura de la celda limpia en Celsius")
        logger.info(f"   - Tp(C): Temperatura de la celda sucia en Celsius")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos del Soiling Kit desde ClickHouse: {e}")
        import traceback
        logger.error(f"Detalles del error:\n{traceback.format_exc()}")
        return False
    finally:
        if client:
            logger.info("Cerrando conexión a Clickhouse...")
            client.close()
            logger.info("Conexión a Clickhouse cerrada")

In [13]:
# 🔋 DESCARGA TEMPERATURA DE MÓDULOS DESDE INFLUXDB
def download_temp_mod_fixed(influx_client, start_date, end_date, output_dir):
    """Descarga y procesa datos de temperatura de módulos (PT100)."""
    logger.info("Iniciando descarga de datos de Temperatura de Módulos (PT100)...")
    
    try:
        bucket = "PSDA"
        attributes = ["1TE416(C)", "1TE417(C)", "1TE418(C)", "1TE419(C)"] # Sensores PT100
        
        # Fuente 1: Tabla TempModFixed
        tables_source1 = ["TempModFixed"]
        logger.info(f"Consultando datos de temperatura de {tables_source1}...")
        df_temp_source1 = influx_client.query_influxdb(bucket, tables_source1, attributes, start_date, end_date)
        
        if df_temp_source1 is not None and not df_temp_source1.empty:
            if '_time' in df_temp_source1.columns:
                df_temp_source1.set_index('_time', inplace=True)
            elif 'time' in df_temp_source1.columns: # Por si acaso el nombre de la columna de tiempo varía
                df_temp_source1.set_index('time', inplace=True)

            if not isinstance(df_temp_source1.index, pd.DatetimeIndex):
                df_temp_source1.index = pd.to_datetime(df_temp_source1.index)
            
            df_temp_source1 = df_temp_source1.between_time('13:00', '18:00') # Filtro horario como en soiling_intercomparison.py
            logger.info(f"Datos de {tables_source1} procesados. Registros: {len(df_temp_source1)}")
        else:
            logger.warning(f"No se obtuvieron datos de temperatura de {tables_source1} o el DataFrame está vacío.")
            df_temp_source1 = pd.DataFrame() # Asegurar que sea un DF vacío si no hay datos

        # Fuente 2: Tabla fixed_plant_atamo_1 (como en soiling_intercomparison.py)
        # El script original usa una fecha de inicio fija para esta fuente: pd.to_datetime('05/12/2024', dayfirst=True)
        # Usaremos la 'start_date' global, pero puedes ajustarla si necesitas la fecha fija.
        # Para replicar exactamente, podrías usar:
        # date_s_fixed_plant = pd.to_datetime('05/12/2024', dayfirst=True).tz_localize('UTC')
        # Y luego pasar date_s_fixed_plant a query_influxdb para esta fuente.
        # Por ahora, usaremos start_date y end_date globales.
        tables_source2 = ["fixed_plant_atamo_1"]
        logger.info(f"Consultando datos de temperatura de {tables_source2}...")
        df_temp_source2 = influx_client.query_influxdb(bucket, tables_source2, attributes, start_date, end_date)

        if df_temp_source2 is not None and not df_temp_source2.empty:
            if '_time' in df_temp_source2.columns:
                df_temp_source2.set_index('_time', inplace=True)
            elif 'time' in df_temp_source2.columns:
                df_temp_source2.set_index('time', inplace=True)

            if not isinstance(df_temp_source2.index, pd.DatetimeIndex):
                df_temp_source2.index = pd.to_datetime(df_temp_source2.index)
                
            df_temp_source2 = df_temp_source2.between_time('13:00', '18:00')
            logger.info(f"Datos de {tables_source2} procesados. Registros: {len(df_temp_source2)}")
        else:
            logger.warning(f"No se obtuvieron datos de temperatura de {tables_source2} o el DataFrame está vacío.")
            df_temp_source2 = pd.DataFrame()

        # Concatenar datos de ambas fuentes
        dataframes_to_concat = []
        if not df_temp_source1.empty:
            dataframes_to_concat.append(df_temp_source1)
        if not df_temp_source2.empty:
            dataframes_to_concat.append(df_temp_source2)

        if not dataframes_to_concat:
            logger.warning("No hay datos de temperatura de ninguna fuente para concatenar.")
            return False
            
        df_temp_combined = pd.concat(dataframes_to_concat)
        df_temp_combined = df_temp_combined.sort_index() # Ordenar por tiempo
        
        # Eliminar columnas que pueden ser resultado de la consulta y no son atributos (result, table)
        cols_to_drop = [col for col in ['result', 'table'] if col in df_temp_combined.columns]
        if cols_to_drop:
            df_temp_combined.drop(columns=cols_to_drop, inplace=True)
            
        # Guardar datos
        output_filepath = os.path.join(output_dir, 'temp_mod_fixed_data.csv')
        df_temp_combined.to_csv(output_filepath)
        
        logger.info(f"Datos de Temperatura de Módulos guardados exitosamente en {output_filepath}")
        logger.info(f"Total de registros combinados: {len(df_temp_combined)}")
        if not df_temp_combined.empty:
            logger.info(f"Rango de fechas: {df_temp_combined.index.min()} a {df_temp_combined.index.max()}")
        
        return True
        
    except Exception as e:
        logger.error(f"Error en la descarga de datos de Temperatura de Módulos: {e}")
        import traceback
        logger.error(f"Detalles del error:\\n{traceback.format_exc()}")
        return False

# 🔋 **REFCELLS - CELDAS SIMPLIFICADAS**

## 🚫 **CELDAS OBSOLETAS (NO USAR):**

### ❌ **Celda 12** - Función básica obsoleta
- Versión antigua de `download_refcells()`
- **REEMPLAZADA** por Celda 17

### ❌ **Celda 14** - Ejecución ya completada  
- Código que ya se ejecutó exitosamente
- **REDUNDANTE** con Celda 17 + 19

### ❌ **Celda 18** - Ejecución automática
- Se ejecuta automáticamente al cargar
- **CAUSA PROBLEMAS** y duplica funcionalidad

---

## ✅ **CELDAS A USAR (SOLO ESTAS 2):**

### 📝 **Celda 17** - Función Avanzada
- `download_refcells()` **optimizada y completa**
- Solo **define** la función (no ejecuta)

### 🚀 **Celda 19** - Ejecución Simple
- **Ejecuta** la función cuando tú quieras
- Configuración automática incluida

---

## 🎯 **INSTRUCCIONES:**
1. **Ejecuta Celda 17** → Define función
2. **Ejecuta Celda 19** → Descarga datos
3. **¡Listo!** → Ignora las demás celdas de RefCells


In [15]:
# DESCARGA DE TODOS LOS DATOS
# ============================================================================

def download_all_data(start_date, end_date, output_dir):
    """Función principal que coordina la descarga de todos los datos."""
    logger.info("Iniciando proceso de descarga de datos...")
    
    # Crear directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Inicializar resultados
    results = {}
    
    try:
        # Primero descargar datos de Clickhouse
        logger.info("\nIniciando descarga de datos desde Clickhouse...")
        results['iv600'] = download_iv600(start_date, end_date, output_dir)
        logger.info(f"Descarga IV600: {'Exitosa' if results['iv600'] else 'Fallida'}")
        
        # Liberar memoria
        gc.collect()
        
        # Luego descargar datos de InfluxDB
        logger.info("\nIniciando descargas desde InfluxDB...")
        
        # Inicializar cliente InfluxDB
        influx_manager = InfluxDBManager(INFLUX_CONFIG)
        if not influx_manager.connect():
            logger.error("No se pudo establecer conexión con InfluxDB")
            return results
            
        try:
            # Descargar datos de PV Glasses
            logger.info("\nProcesando PV Glasses...")
            results['pv_glasses'] = download_pv_glasses(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Descarga PV Glasses: {'Exitosa' if results['pv_glasses'] else 'Fallida'}")
            
            # Descargar datos de DustIQ
            logger.info("\nProcesando DustIQ...")
            results['dustiq'] = download_dustiq(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Descarga DustIQ: {'Exitosa' if results['dustiq'] else 'Fallida'}")
            
            # Descargar datos de PVStand
            logger.info("\nProcesando PVStand...")
            results['pvstand'] = download_pvstand(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Descarga PVStand: {'Exitosa' if results['pvstand'] else 'Fallida'}")
            
            # Descargar datos de celdas de referencia (CON FECHA ESPECÍFICA Y PROCESAMIENTO)
            logger.info("\nProcesando celdas de referencia...")
            # Fecha específica para refcells: 23 de julio de 2024
            refcells_start_date = pd.to_datetime('23/07/2024', dayfirst=True).tz_localize('UTC')
            logger.info(f"Usando fecha específica para RefCells: {refcells_start_date}")
            
            results['refcells'] = download_refcells(influx_manager, refcells_start_date, end_date, output_dir)
            logger.info(f"Descarga celdas de referencia: {'Exitosa' if results['refcells'] else 'Fallida'}")
            
            # PROCESAMIENTO ADICIONAL DE REFCELLS
            if results['refcells']:
                logger.info("Aplicando procesamiento adicional a RefCells...")
                try:
                    # Leer el archivo generado
                    input_filepath = os.path.join(output_dir, 'refcells_data.csv')
                    df = pd.read_csv(input_filepath)
                    
                    # Establecer _time como índice
                    df['_time'] = pd.to_datetime(df['_time'])
                    df = df.set_index('_time')
                    
                    # Resample por minuto (promedio)
                    logger.info("Aplicando resample por minuto a RefCells...")
                    df_resampled = df.resample('1min').mean()
                    df_resampled = df_resampled.dropna(how='all')
                    
                    # Sobrescribir el archivo original con los datos procesados
                    final_output_path = os.path.join(output_dir, 'refcells_data.csv')
                    df_resampled.to_csv(final_output_path)
                    
                    logger.info(f"RefCells procesadas: {len(df)} → {len(df_resampled)} registros")
                    results['refcells_processed'] = True
                    
                except Exception as e:
                    logger.error(f"Error en procesamiento de RefCells: {e}")
                    results['refcells_processed'] = False
            
            # Descargar datos del Soiling Kit
            logger.info("\nProcesando Soiling Kit...")
            results['soiling_kit'] = download_soiling_kit(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Descarga Soiling Kit: {'Exitosa' if results['soiling_kit'] else 'Fallida'}")

            # Descargar datos de Temperatura de Módulos
            logger.info("\nProcesando Temperatura de Módulos (PT100)...")
            results['temp_mod_fixed'] = download_temp_mod_fixed(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Descarga Temperatura de Módulos: {'Exitosa' if results['temp_mod_fixed'] else 'Fallida'}")

        finally:
            # Cerrar conexión con InfluxDB
            influx_manager.disconnect()
            
        # Resumen de resultados
        logger.info("\nResumen de descargas:")
        for key, value in results.items():
            logger.info(f"{key}: {'Exitosa' if value else 'Fallida'}")
            
        return results
        
    except Exception as e:
        logger.error(f"Error en el proceso de descarga: {e}")
        return results

In [ ]:
# 🌪️ DESCARGA ACTUALIZADA: SOILING KIT CON VERIFICACIÓN PREVIA DESDE INFLUXDB
# ============================================================================
# Descarga más ineficiente que la de Clickhouse, pero se mantiene para compatibilidad con el código de soiling_intercomparison.py
# VERSIÓN MEJORADA: Incluye verificación de datos disponibles antes de descargar

logger.info("\n" + "="*80)
logger.info("🌪️ DESCARGA ACTUALIZADA: SOILING KIT CON VERIFICACIÓN")
logger.info("="*80)

# Configurar fechas: desde julio 2024 hasta la actualidad
START_DATE_SK = pd.to_datetime('01/07/2024', dayfirst=True).tz_localize('UTC')  # Desde julio 2024
END_DATE_SK = pd.to_datetime('31/07/2025', dayfirst=True).tz_localize('UTC')    # Hasta julio 2025 (actualidad)

logger.info(f"📅 Rango actualizado: {START_DATE_SK.strftime('%Y-%m-%d')} a {END_DATE_SK.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")

# Inicializar cliente InfluxDB
influx_manager = InfluxDBManager(INFLUX_CONFIG)

try:
    if not influx_manager.connect():
        logger.error("❌ No se pudo establecer conexión con InfluxDB")
    else:
        logger.info("✅ Conexión a InfluxDB establecida")
        
        # ===== VERIFICACIÓN PREVIA DE DATOS DISPONIBLES =====
        logger.info("\n🔍 VERIFICANDO DATOS DISPONIBLES ANTES DE DESCARGAR...")
        
        # Consulta para verificar datos disponibles en el rango especificado
        verification_query = f'''
        from(bucket: "PSDA")
            |> range(start: {START_DATE_SK.strftime("%Y-%m-%dT%H:%M:%SZ")}, stop: {END_DATE_SK.strftime("%Y-%m-%dT%H:%M:%SZ")})
            |> filter(fn: (r) => r["_measurement"] == "soilingkit")
            |> filter(fn: (r) => r["_field"] == "Isc(e)" or r["_field"] == "Isc(p)" or r["_field"] == "Te(C)" or r["_field"] == "Tp(C)")
            |> keep(columns: ["_time", "_field", "_value"])
            |> sort(columns: ["_time"])
        '''
        
        try:
            logger.info("🔍 Ejecutando consulta de verificación...")
            verification_result = influx_manager.query_api.query_data_frame(verification_query)
            
            if verification_result.empty:
                logger.warning("⚠️ NO SE ENCONTRARON DATOS en el rango especificado")
                logger.info("🔍 Intentando verificar datos disponibles en un rango más amplio...")
                
                # Consulta más amplia para ver qué datos están disponibles
                wide_query = f'''
                from(bucket: "PSDA")
                    |> range(start: 2024-01-01T00:00:00Z, stop: 2025-12-31T23:59:59Z)
                    |> filter(fn: (r) => r["_measurement"] == "soilingkit")
                    |> filter(fn: (r) => r["_field"] == "Isc(e)" or r["_field"] == "Isc(p)" or r["_field"] == "Te(C)" or r["_field"] == "Tp(C)")
                    |> keep(columns: ["_time", "_field", "_value"])
                    |> sort(columns: ["_time"])
                '''
                
                wide_result = influx_manager.query_api.query_data_frame(wide_query)
                
                if not wide_result.empty:
                    logger.info(f"📊 Datos disponibles en rango amplio: {len(wide_result)} registros")
                    logger.info(f"📅 Fecha más antigua disponible: {wide_result['_time'].min()}")
                    logger.info(f"📅 Fecha más reciente disponible: {wide_result['_time'].max()}")
                    
                    # Analizar por campo
                    if '_field' in wide_result.columns:
                        field_counts = wide_result['_field'].value_counts()
                        logger.info("📊 Registros por campo:")
                        for field, count in field_counts.items():
                            logger.info(f"   - {field}: {count} registros")
                    
                    logger.warning("⚠️ El rango de fechas especificado no tiene datos")
                    logger.info("💡 Considera ajustar las fechas o usar el rango disponible")
                    
                else:
                    logger.error("❌ No se encontraron datos de Soiling Kit en ningún rango")
                    logger.error("🔧 Verifica la configuración de InfluxDB y el bucket PSDA")
                
                # No proceder con la descarga
                success = False
                
            else:
                # Datos encontrados, mostrar información
                logger.info(f"✅ DATOS ENCONTRADOS: {len(verification_result)} registros")
                logger.info(f"📅 Fecha más antigua: {verification_result['_time'].min()}")
                logger.info(f"📅 Fecha más reciente: {verification_result['_time'].max()}")
                
                # Analizar por campo
                if '_field' in verification_result.columns:
                    field_counts = verification_result['_field'].value_counts()
                    logger.info("📊 Registros por campo:")
                    for field, count in field_counts.items():
                        logger.info(f"   - {field}: {count} registros")
                
                # Mostrar estadísticas por mes
                verification_result['_time'] = pd.to_datetime(verification_result['_time'])
                verification_result['year_month'] = verification_result['_time'].dt.to_period('M')
                monthly_counts = verification_result.groupby('year_month').size()
                
                logger.info("\n📈 Registros por mes:")
                for month, count in monthly_counts.items():
                    logger.info(f"   - {month}: {count} registros")
                
                logger.info("✅ Verificación completada. Procediendo con la descarga...")
                
                # ===== EJECUTAR DESCARGA DEL SOILING KIT =====
                logger.info("\n🌪️ Descargando datos actualizados del Soiling Kit...")
                success = download_soiling_kit(influx_manager, START_DATE_SK, END_DATE_SK, OUTPUT_DIR)
                
        except Exception as e:
            logger.error(f"❌ Error en la verificación: {e}")
            logger.info("⚠️ Procediendo con descarga sin verificación...")
            success = download_soiling_kit(influx_manager, START_DATE_SK, END_DATE_SK, OUTPUT_DIR)
        
        # ===== VERIFICAR RESULTADO DE LA DESCARGA =====
        if success:
            # Verificar archivo generado
            output_file = os.path.join(OUTPUT_DIR, 'soiling_kit_raw_data.csv')
            if os.path.exists(output_file):
                file_size_mb = os.path.getsize(output_file) / (1024*1024)
                total_lines = sum(1 for line in open(output_file)) - 1
                
                logger.info("\n" + "="*60)
                logger.info("🎉 ¡DESCARGA ACTUALIZADA COMPLETADA!")
                logger.info(f"📂 Archivo: soiling_kit_raw_data.csv")
                logger.info(f"📊 Tamaño: {file_size_mb:.2f} MB")
                logger.info(f"📈 Total de registros: {total_lines:,}")
                logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
                logger.info("🌪️ Datos incluyen: Isc(e), Isc(p), Te(C), Tp(C)")
                logger.info("📅 Período: Julio 2024 - Julio 2025")
                logger.info("="*60)
                
                # Mostrar muestra de los datos
                try:
                    df_sample = pd.read_csv(output_file, nrows=5)
                    logger.info("\n📋 Muestra de los datos:")
                    logger.info(f"   Columnas: {list(df_sample.columns)}")
                    if 'Isc(e)' in df_sample.columns:
                        logger.info(f"   Isc(e) rango: {df_sample['Isc(e)'].min():.3f} - {df_sample['Isc(e)'].max():.3f}")
                    if 'Isc(p)' in df_sample.columns:
                        logger.info(f"   Isc(p) rango: {df_sample['Isc(p)'].min():.3f} - {df_sample['Isc(p)'].max():.3f}")
                except Exception as e:
                    logger.warning(f"⚠️ Error al mostrar muestra: {e}")
            else:
                logger.warning("⚠️ Archivo no encontrado después de la descarga")
        else:
            logger.error("❌ Error en la descarga del Soiling Kit")
            
except Exception as e:
    logger.error(f"❌ Error general: {e}")
    import traceback
    logger.error(f"🔍 Detalles:\n{traceback.format_exc()}")
    
finally:
    influx_manager.disconnect()
    logger.info("🔌 Conexión cerrada")

logger.info("\n🏁 DESCARGA ACTUALIZADA CON VERIFICACIÓN COMPLETADA") 

2025-08-06 12:40:59 - INFO - 
2025-08-06 12:40:59 - INFO - 🌪️ DESCARGA ACTUALIZADA: SOILING KIT CON VERIFICACIÓN
2025-08-06 12:40:59 - INFO - ================================================================================
2025-08-06 12:41:00 - INFO - 📅 Rango actualizado: 2024-07-01 a 2025-07-31
2025-08-06 12:41:00 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-08-06 12:41:02 - INFO - Cliente InfluxDB y query_api inicializados.
2025-08-06 12:41:02 - INFO - ✅ Conexión a InfluxDB establecida
2025-08-06 12:41:02 - INFO - 
🔍 VERIFICANDO DATOS DISPONIBLES ANTES DE DESCARGAR...
2025-08-06 12:41:02 - INFO - 🔍 Ejecutando consulta de verificación...
/home/nicole/SR/SOILING/.venv/lib/python3.12/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
        from(bucket: "PSDA")
            |> range(st

In [44]:
# === EJECUCIÓN PRINCIPAL: DESCARGA COMPLETA DE TODOS LOS DATOS ===

logger.info("\n" + "="*80)
logger.info("🚀 INICIANDO DESCARGA COMPLETA DE TODOS LOS DATOS")
logger.info("="*80)

logger.info(f"📅 Rango general: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info(f"📂 RefCells usará fecha específica: 2024-07-23")

# Ejecutar descarga completa
results = download_all_data(START_DATE, END_DATE, OUTPUT_DIR)

# Resumen final detallado
logger.info("\n" + "="*80)
logger.info("📊 RESUMEN FINAL DE DESCARGAS")
logger.info("="*80)

total_successful = sum(1 for v in results.values() if v)
total_processes = len(results)

logger.info(f"✅ Procesos exitosos: {total_successful}/{total_processes}")

# Detalle por proceso
status_emoji = lambda x: "✅" if x else "❌"
for process, success in results.items():
    process_name = {
        'iv600': 'IV600 (Clickhouse)',
        'pv_glasses': 'PV Glasses',
        'dustiq': 'DustIQ', 
        'pvstand': 'PVStand',
        'refcells': 'RefCells (Descarga)',
        'refcells_processed': 'RefCells (Procesamiento)',
        'soiling_kit': 'Soiling Kit',
        'temp_mod_fixed': 'Temperatura Módulos'
    }.get(process, process)
    
    logger.info(f"{status_emoji(success)} {process_name}")

if total_successful == total_processes:
    logger.info("\n🎉 ¡TODOS LOS PROCESOS COMPLETADOS EXITOSAMENTE!")
    logger.info("📂 Archivos generados en: " + OUTPUT_DIR)
    logger.info("🔍 RefCells procesado con resample 1min y guardado como refcells_data.csv")
else:
    logger.warning(f"\n⚠️ {total_processes - total_successful} proceso(s) fallaron")
    logger.info("🔧 Revisa los logs anteriores para detalles de errores")

logger.info("="*80)
logger.info("🏁 PROCESO COMPLETO FINALIZADO")
logger.info("="*80)

2025-06-19 11:47:24 - INFO - 
2025-06-19 11:47:24 - INFO - 🚀 INICIANDO DESCARGA COMPLETA DE TODOS LOS DATOS
2025-06-19 11:47:24 - INFO - ================================================================================
2025-06-19 11:47:24 - INFO - 📅 Rango general: 2024-07-01 a 2025-12-31
2025-06-19 11:47:24 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-06-19 11:47:24 - INFO - 📂 RefCells usará fecha específica: 2024-07-23
2025-06-19 11:47:24 - INFO - Iniciando proceso de descarga de datos...
2025-06-19 11:47:24 - INFO - 
Iniciando descarga de datos desde Clickhouse...
2025-06-19 11:47:24 - INFO - Iniciando descarga de datos IV600...
2025-06-19 11:47:24 - INFO - Conectando a Clickhouse...


2025-06-19 11:47:25 - INFO - Conexión a Clickhouse establecida
2025-06-19 11:47:25 - INFO - Consultando datos IV600...
2025-06-19 11:47:26 - INFO - Datos obtenidos: 2029 registros
2025-06-19 11:47:26 - INFO - Procesando datos...
2025-06-19 11:47:26 - INFO - Creando DataFrame...
2025-06-19 11:47:26 - INFO - Rango de fechas en los datos:
2025-06-19 11:47:26 - INFO - Fecha más antigua: 2024-09-24 12:16:00+00:00
2025-06-19 11:47:26 - INFO - Fecha más reciente: 2025-04-22 12:43:00+00:00
2025-06-19 11:47:26 - INFO - Filtrando datos entre 2024-07-01 00:00:00+00:00 y 2025-12-31 00:00:00+00:00...
2025-06-19 11:47:26 - INFO - Se encontraron 2029 registros en el rango especificado.
2025-06-19 11:47:26 - INFO - Guardando datos en: /home/nicole/SR/SOILING/datos/raw_iv600_data.csv
2025-06-19 11:47:26 - INFO - Datos guardados exitosamente. Total de registros: 2029
2025-06-19 11:47:26 - INFO - Rango de fechas: 2024-09-24 12:16:00+00:00 a 2025-04-22 12:43:00+00:00
2025-06-19 11:47:26 - INFO - Cerrando 

: 

# Para descargar todo de una vez

2025-08-07 12:12:24 - INFO - ✅ Función download_refcells() definida correctamente


In [2]:
def download_all_data_optimized(start_date, end_date, output_dir):
    """Función principal optimizada para manejo de memoria."""
    import gc
    
    logger.info("Iniciando proceso de descarga optimizado...")
    os.makedirs(output_dir, exist_ok=True)
    results = {}
    
    try:
        # IV600 (Clickhouse)
        logger.info("\n🔹 Descargando IV600...")
        gc.collect()
        results['iv600'] = download_iv600(start_date, end_date, output_dir)
        logger.info(f"IV600: {'✅' if results['iv600'] else '❌'}")
        gc.collect()
        
        # InfluxDB Manager - reutilizar conexión pero limpiar datos
        influx_manager = InfluxDBManager(INFLUX_CONFIG)
        if not influx_manager.connect():
            return results
            
        try:
            # PV Glasses
            logger.info("\n🔹 Descargando PV Glasses...")
            gc.collect()
            results['pv_glasses'] = download_pv_glasses(influx_manager, start_date, end_date, output_dir)
            logger.info(f"PV Glasses: {'✅' if results['pv_glasses'] else '❌'}")
            gc.collect()
            
            # DustIQ
            logger.info("\n🔹 Descargando DustIQ...")
            gc.collect()
            results['dustiq'] = download_dustiq(influx_manager, start_date, end_date, output_dir)
            logger.info(f"DustIQ: {'✅' if results['dustiq'] else '❌'}")
            gc.collect()
            
            # PVStand
            logger.info("\n🔹 Descargando PVStand...")
            gc.collect()
            results['pvstand'] = download_pvstand(influx_manager, start_date, end_date, output_dir)
            logger.info(f"PVStand: {'✅' if results['pvstand'] else '❌'}")
            gc.collect()
            
            # RefCells con procesamiento optimizado
            logger.info("\n🔹 Descargando RefCells...")
            refcells_start = pd.to_datetime('23/07/2024', dayfirst=True).tz_localize('UTC')
            gc.collect()
            results['refcells'] = download_refcells(influx_manager, refcells_start, end_date, output_dir)
            logger.info(f"RefCells Descarga: {'✅' if results['refcells'] else '❌'}")
            
            # Procesamiento RefCells
            if results['refcells']:
                logger.info("🔹 Procesando RefCells...")
                try:
                    input_filepath = os.path.join(output_dir, 'refcells_data.csv')
                    
                    # Procesar en chunks
                    chunks = []
                    for chunk in pd.read_csv(input_filepath, chunksize=30000):
                        chunk['_time'] = pd.to_datetime(chunk['_time'])
                        chunk = chunk.set_index('_time')
                        chunks.append(chunk)
                    
                    df = pd.concat(chunks)
                    del chunks
                    gc.collect()
                    
                    df_resampled = df.resample('1min').mean().dropna(how='all')
                    del df
                    gc.collect()
                    
                    df_resampled.to_csv(os.path.join(output_dir, 'refcells_data.csv'))
                    results['refcells_processed'] = True
                    logger.info(f"RefCells Procesamiento: ✅ ({len(df_resampled)} registros)")
                    del df_resampled
                    gc.collect()
                    
                except Exception as e:
                    logger.error(f"Error procesando RefCells: {e}")
                    results['refcells_processed'] = False
            
            # Soiling Kit
            logger.info("\n🔹 Descargando Soiling Kit...")
            gc.collect()
            results['soiling_kit'] = download_soiling_kit(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Soiling Kit: {'✅' if results['soiling_kit'] else '❌'}")
            gc.collect()
            
            # Temperatura
            logger.info("\n🔹 Descargando Temperatura...")
            gc.collect()
            results['temp_mod_fixed'] = download_temp_mod_fixed(influx_manager, start_date, end_date, output_dir)
            logger.info(f"Temperatura: {'✅' if results['temp_mod_fixed'] else '❌'}")
            gc.collect()
            
        finally:
            influx_manager.disconnect()
            
        # Resumen
        successful = sum(1 for v in results.values() if v)
        logger.info(f"\n🎉 Completado: {successful}/{len(results)} exitosos")
        return results
        
    except Exception as e:
        logger.error(f"Error general: {e}")
        return results

In [15]:
# 🚀 **EJECUTAR DESCARGA DE REFCELLS** - CELDA SIMPLE
# ============================================================================
# Esta es la ÚNICA celda que debes ejecutar para descargar RefCells

logger.info("🔋 Iniciando descarga de RefCells...")

# Configurar fechas
refcells_start_date = pd.to_datetime('23/07/2024', dayfirst=True).tz_localize('UTC')
end_date = pd.to_datetime('31/12/2025', dayfirst=True).tz_localize('UTC')

# Inicializar cliente InfluxDB
influx_manager = InfluxDBManager(INFLUX_CONFIG)

try:
    if not influx_manager.connect():
        logger.error("❌ No se pudo conectar a InfluxDB")
    else:
        logger.info("✅ Conexión establecida")
        
        # 🎯 EJECUTAR LA FUNCIÓN (definida en Celda 17)
        success = download_refcells(influx_manager, refcells_start_date, end_date, OUTPUT_DIR)
        
        if success:
            logger.info("🎉 ¡Descarga completada exitosamente!")
        else:
            logger.error("❌ Error en la descarga")
            
finally:
    influx_manager.disconnect()
    logger.info("🔌 Conexión cerrada")

logger.info("🏁 Proceso finalizado")


2025-08-07 12:12:35 - INFO - 🔋 Iniciando descarga de RefCells...
2025-08-07 12:12:35 - INFO - Cliente InfluxDB y query_api inicializados.
2025-08-07 12:12:35 - INFO - ✅ Conexión establecida
2025-08-07 12:12:35 - INFO - 🔋 Iniciando descarga optimizada de datos de celdas de referencia...
2025-08-07 12:12:35 - INFO - 🚀 PASO 1: DESCARGANDO DATOS (OPTIMIZADO)...
2025-08-07 12:12:35 - INFO - 📊 Consultando InfluxDB: bucket=PSDA, tables=['fixed_plant_atamo_1'], attributes=['1RC410(w.m-2)', '1RC411(w.m-2)', '1RC412(w.m-2)']
2025-08-07 12:12:35 - INFO - Consultando InfluxDB: bucket=PSDA, tables=['fixed_plant_atamo_1'], attributes=['1RC410(w.m-2)', '1RC411(w.m-2)', '1RC412(w.m-2)']
2025-08-07 12:13:39 - INFO - 📊 Datos obtenidos: 1,062,471 registros
2025-08-07 12:13:39 - INFO - 📊 Columnas originales: ['result', 'table', '_start', '_stop', '_time', '_measurement', '1RC410(w.m-2)', '1RC411(w.m-2)', '1RC412(w.m-2)']
2025-08-07 12:13:39 - INFO - 🔄 PASO 2: PROCESANDO FORMATO DE DATOS...
2025-08-07 12:1

In [14]:
# === EJECUCIÓN OPTIMIZADA COMPLETA ===
logger.info("🚀 Iniciando descarga completa optimizada...")
results = download_all_data_optimized(START_DATE, END_DATE, OUTPUT_DIR)

# Resumen final
logger.info("\n" + "="*50)
for process, success in results.items():
    status = "✅" if success else "❌"
    logger.info(f"{status} {process}")
logger.info("="*50)

2025-06-19 11:58:17 - INFO - 🚀 Iniciando descarga completa optimizada...
2025-06-19 11:58:17 - INFO - Iniciando proceso de descarga optimizado...
2025-06-19 11:58:17 - INFO - 
🔹 Descargando IV600...
2025-06-19 11:58:17 - INFO - Iniciando descarga de datos IV600...
2025-06-19 11:58:17 - INFO - Conectando a Clickhouse...
2025-06-19 11:58:17 - INFO - Conexión a Clickhouse establecida
2025-06-19 11:58:17 - INFO - Consultando datos IV600...
2025-06-19 11:58:18 - INFO - Datos obtenidos: 2029 registros
2025-06-19 11:58:18 - INFO - Procesando datos...
2025-06-19 11:58:18 - INFO - Creando DataFrame...
2025-06-19 11:58:18 - INFO - Rango de fechas en los datos:
2025-06-19 11:58:18 - INFO - Fecha más antigua: 2024-09-24 12:16:00+00:00
2025-06-19 11:58:18 - INFO - Fecha más reciente: 2025-04-22 12:43:00+00:00
2025-06-19 11:58:18 - INFO - Filtrando datos entre 2024-07-01 00:00:00+00:00 y 2025-12-31 00:00:00+00:00...
2025-06-19 11:58:18 - INFO - Se encontraron 2029 registros en el rango especificado.

In [ ]:
# 🚀 EJECUTAR SOLO DESCARGA PVSTAND DESDE CLICKHOUSE
# ============================================================================

logger.info("\n" + "="*80)
logger.info("🔋 DESCARGA PVSTAND DESDE CLICKHOUSE")
logger.info("="*80)

logger.info(f"�� Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info("🚀 Descarga desde ClickHouse - PSDA.perc1fixed y PSDA.perc2fixed")

# Ejecutar descarga de PVStand desde ClickHouse
success = download_pvstand_clickhouse(START_DATE, END_DATE, OUTPUT_DIR)

if success:
    # Verificar archivo generado
    output_file = os.path.join(OUTPUT_DIR, 'raw_pvstand_iv_data.csv')
    if os.path.exists(output_file):
        file_size_mb = os.path.getsize(output_file) / (1024*1024)
        total_lines = sum(1 for line in open(output_file)) - 1
        
        logger.info("\n" + "="*60)
        logger.info("🎉 ¡DESCARGA PVSTAND DESDE CLICKHOUSE COMPLETADA!")
        logger.info(f"📂 Archivo: raw_pvstand_iv_data.csv")
        logger.info(f"�� Tamaño: {file_size_mb:.2f} MB")
        logger.info(f"�� Total de registros: {total_lines:,}")
        logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
        logger.info("�� Datos incluyen: pmax, imax, umax")
        logger.info("�� Período: Julio 2024 - Julio 2025")
        logger.info("="*60)
        
        # Mostrar muestra de los datos
        try:
            df_sample = pd.read_csv(output_file, nrows=5)
            logger.info("\n📋 Muestra de los datos:")
            logger.info(f"   Columnas: {list(df_sample.columns)}")
            if 'pmax' in df_sample.columns:
                logger.info(f"   pmax rango: {df_sample['pmax'].min():.3f} - {df_sample['pmax'].max():.3f}")
            if 'imax' in df_sample.columns:
                logger.info(f"   imax rango: {df_sample['imax'].min():.3f} - {df_sample['imax'].max():.3f}")
            if 'umax' in df_sample.columns:
                logger.info(f"   umax rango: {df_sample['umax'].min():.3f} - {df_sample['umax'].max():.3f}")
        except Exception as e:
            logger.warning(f"⚠️ Error al mostrar muestra: {e}")
    else:
        logger.warning("⚠️ Archivo no encontrado después de la descarga")
else:
    logger.error("❌ Error en la descarga de PVStand desde ClickHouse")

logger.info("\n🏁 DESCARGA PVSTAND DESDE CLICKHOUSE COMPLETADA")

2025-07-29 10:17:47 - INFO - 
2025-07-29 10:17:47 - INFO - 🔋 DESCARGA PVSTAND DESDE CLICKHOUSE
2025-07-29 10:17:47 - INFO - ================================================================================
2025-07-29 10:17:47 - INFO - �� Rango de fechas: 2024-07-01 a 2025-12-31
2025-07-29 10:17:47 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-07-29 10:17:47 - INFO - 🚀 Descarga desde ClickHouse - PSDA.perc1fixed y PSDA.perc2fixed
2025-07-29 10:17:47 - INFO - Iniciando descarga de datos PVStand desde ClickHouse...
2025-07-29 10:17:47 - INFO - Conectando a Clickhouse...
2025-07-29 10:17:47 - INFO - Conexión a Clickhouse establecida
2025-07-29 10:17:47 - INFO - Consultando datos PVStand desde ClickHouse...
2025-07-29 10:17:47 - INFO - Ejecutando consulta: 
        SELECT 
            timestamp,
            'perc1fixed' as module,
            pmax,
      ...
2025-07-29 10:17:50 - INFO - Datos obtenidos: 221608 registros
2025-07-29 10:17:50 - INFO - Procesando datos...
2

In [ ]:
# 🔋 DESCARGA ESPECÍFICA: SOLO DATOS DE PVSTAND ACTUALIZADOS INFLUXDB
# ============================================================================

logger.info("\n" + "="*80)
logger.info("🔋 INICIANDO DESCARGA ESPECÍFICA: DATOS DE PVSTAND")
logger.info("="*80)

logger.info(f"📅 Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info("🚀 Descarga optimizada solo para PVStand")

# Inicializar cliente InfluxDB
influx_manager = InfluxDBManager(INFLUX_CONFIG)

try:
    if not influx_manager.connect():
        logger.error("❌ No se pudo establecer conexión con InfluxDB")
    else:
        logger.info("✅ Conexión a InfluxDB establecida")
        
        # Ejecutar descarga específica de PVStand
        logger.info("\n🔋 Iniciando descarga de PVStand...")
        success = download_pvstand(influx_manager, START_DATE, END_DATE, OUTPUT_DIR)
        
        # Verificar resultado y mostrar información del archivo
        if success:
            output_file = os.path.join(OUTPUT_DIR, 'raw_pvstand_iv_data.csv')
            if os.path.exists(output_file):
                file_size_mb = os.path.getsize(output_file) / (1024*1024)
                
                # Leer primeras líneas para verificar datos
                try:
                    df_check = pd.read_csv(output_file, nrows=5)
                    logger.info(f"📊 Columnas en el archivo: {list(df_check.columns)}")
                    logger.info(f"📈 Primeros registros: {len(df_check)}")
                    
                    # Contar total de registros
                    total_lines = sum(1 for line in open(output_file)) - 1  # -1 para header
                    
                    logger.info("\n" + "="*60)
                    logger.info("🎉 ¡DESCARGA DE PVSTAND COMPLETADA!")
                    logger.info(f"📂 Archivo: raw_pvstand_iv_data.csv")
                    logger.info(f"📊 Tamaño: {file_size_mb:.2f} MB")
                    logger.info(f"📈 Total de registros: {total_lines:,}")
                    logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
                    logger.info("="*60)
                    
                except Exception as e:
                    logger.warning(f"⚠️ Error al verificar contenido del archivo: {e}")
                    logger.info("✅ Archivo generado exitosamente")
            else:
                logger.warning("⚠️ Archivo no encontrado después de la descarga")
        else:
            logger.error("\n" + "="*50)
            logger.error("❌ ERROR EN LA DESCARGA DE PVSTAND")
            logger.error("🔍 Revisa los logs anteriores para más detalles")
            logger.error("🔧 Verifica la conexión y configuración de InfluxDB")
            logger.error("="*50)

except Exception as e:
    logger.error(f"❌ Error general en el proceso: {e}")
    import traceback
    logger.error(f"🔍 Detalles del error:\n{traceback.format_exc()}")
    
finally:
    # Cerrar conexión
    influx_manager.disconnect()
    logger.info("🔌 Conexión a InfluxDB cerrada")

logger.info("\n🏁 PROCESO DE DESCARGA ESPECÍFICA DE PVSTAND FINALIZADO")


2025-07-29 09:19:02 - INFO - 
2025-07-29 09:19:02 - INFO - 🔋 INICIANDO DESCARGA ESPECÍFICA: DATOS DE PVSTAND
2025-07-29 09:19:02 - INFO - ================================================================================
2025-07-29 09:19:02 - INFO - 📅 Rango de fechas: 2024-07-01 a 2025-12-31
2025-07-29 09:19:02 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-07-29 09:19:02 - INFO - 🚀 Descarga optimizada solo para PVStand
2025-07-29 09:19:02 - INFO - Cliente InfluxDB y query_api inicializados.
2025-07-29 09:19:02 - INFO - ✅ Conexión a InfluxDB establecida
2025-07-29 09:19:02 - INFO - 
🔋 Iniciando descarga de PVStand...
2025-07-29 09:19:02 - ERROR - ❌ Error general en el proceso: name 'download_pvstand' is not defined
2025-07-29 09:19:02 - ERROR - 🔍 Detalles del error:
Traceback (most recent call last):
  File "/tmp/ipykernel_3612/427214555.py", line 24, in <module>
    success = download_pvstand(influx_manager, START_DATE, END_DATE, OUTPUT_DIR)
              ^^^^^^^^^^

In [ ]:
# 📊 DESCARGA ESPECÍFICA: SOLO DATOS DE DUSTIQ ACTUALIZADOS DESDE INFLUXDB
# ============================================================================

logger.info("\n" + "="*80)
logger.info("📊 INICIANDO DESCARGA ESPECÍFICA: DATOS DE DUSTIQ")
logger.info("="*80)

logger.info(f"📅 Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info("🚀 Descarga optimizada solo para DustIQ")

# Inicializar cliente InfluxDB
influx_manager = InfluxDBManager(INFLUX_CONFIG)

try:
    if not influx_manager.connect():
        logger.error("❌ No se pudo establecer conexión con InfluxDB")
    else:
        logger.info("✅ Conexión a InfluxDB establecida")
        
        # Ejecutar descarga específica de DustIQ
        logger.info("\n📊 Iniciando descarga de DustIQ...")
        success = download_dustiq(influx_manager, START_DATE, END_DATE, OUTPUT_DIR)
        
        # Verificar resultado y mostrar información del archivo
        if success:
            output_file = os.path.join(OUTPUT_DIR, 'raw_dustiq_data.csv')
            if os.path.exists(output_file):
                file_size_mb = os.path.getsize(output_file) / (1024*1024)
                
                # Leer primeras líneas para verificar datos
                try:
                    df_check = pd.read_csv(output_file, nrows=10)
                    logger.info(f"📊 Columnas en el archivo: {list(df_check.columns)}")
                    logger.info(f"📈 Primeros registros: {len(df_check)}")
                    
                    # Contar total de registros
                    total_lines = sum(1 for line in open(output_file)) - 1  # -1 para header
                    
                    # Mostrar información específica del DustIQ
                    if 'SR_C11_Avg' in df_check.columns and 'SR_C12_Avg' in df_check.columns:
                        logger.info("🔍 Datos del DustIQ detectados:")
                        logger.info(f"   - SR_C11_Avg - Sensor 1: ✅")
                        logger.info(f"   - SR_C12_Avg - Sensor 2: ✅")
                        
                        # Verificar si hay datos válidos (no NaN)
                        valid_c11 = df_check['SR_C11_Avg'].notna().sum()
                        valid_c12 = df_check['SR_C12_Avg'].notna().sum()
                        logger.info(f"   - Datos válidos SR_C11_Avg: {valid_c11}/{len(df_check)}")
                        logger.info(f"   - Datos válidos SR_C12_Avg: {valid_c12}/{len(df_check)}")
                    
                    logger.info("\n" + "="*60)
                    logger.info("🎉 ¡DESCARGA DE DUSTIQ COMPLETADA!")
                    logger.info(f"📂 Archivo: raw_dustiq_data.csv")
                    logger.info(f"📊 Tamaño: {file_size_mb:.2f} MB")
                    logger.info(f"📈 Total de registros: {total_lines:,}")
                    logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
                    logger.info("📊 Datos incluyen: SR_C11_Avg, SR_C12_Avg")
                    logger.info("📡 Sensores de irradiancia para análisis de soiling")
                    logger.info("="*60)
                    
                except Exception as e:
                    logger.warning(f"⚠️ Error al verificar contenido del archivo: {e}")
                    logger.info("✅ Archivo generado exitosamente")
            else:
                logger.warning("⚠️ Archivo no encontrado después de la descarga")
        else:
            logger.error("\n" + "="*50)
            logger.error("❌ ERROR EN LA DESCARGA DE DUSTIQ")
            logger.error("🔍 Revisa los logs anteriores para más detalles")
            logger.error("🔧 Verifica la conexión y configuración de InfluxDB")
            logger.error("🔗 Bucket: PSDA, Table: DustIQ")
            logger.error("📊 Atributos: SR_C11_Avg, SR_C12_Avg")
            logger.error("="*50)

except Exception as e:
    logger.error(f"❌ Error general en el proceso: {e}")
    import traceback
    logger.error(f"🔍 Detalles del error:\n{traceback.format_exc()}")
    
finally:
    # Cerrar conexión
    influx_manager.disconnect()
    logger.info("🔌 Conexión a InfluxDB cerrada")

logger.info("\n🏁 PROCESO DE DESCARGA ESPECÍFICA DE DUSTIQ FINALIZADO")


2025-07-24 12:54:18 - INFO - 
2025-07-24 12:54:18 - INFO - 📊 INICIANDO DESCARGA ESPECÍFICA: DATOS DE DUSTIQ
2025-07-24 12:54:18 - INFO - ================================================================================
2025-07-24 12:54:18 - INFO - 📅 Rango de fechas: 2024-07-01 a 2025-12-31
2025-07-24 12:54:18 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-07-24 12:54:18 - INFO - 🚀 Descarga optimizada solo para DustIQ
2025-07-24 12:54:18 - INFO - Cliente InfluxDB y query_api inicializados.
2025-07-24 12:54:18 - INFO - ✅ Conexión a InfluxDB establecida
2025-07-24 12:54:18 - INFO - 
📊 Iniciando descarga de DustIQ...
2025-07-24 12:54:18 - INFO - Iniciando descarga de datos DustIQ...
2025-07-24 12:54:18 - INFO - Consultando InfluxDB: bucket=PSDA, tables=['DustIQ'], attributes=['SR_C11_Avg', 'SR_C12_Avg']
2025-07-24 12:54:57 - INFO - Datos DustIQ guardados exitosamente
2025-07-24 12:54:57 - INFO - Total de registros: 537765
2025-07-24 12:54:57 - INFO - Rango de fechas: 0 

In [20]:
download_dustiq_clickhouse(START_DATE, END_DATE, OUTPUT_DIR)

2025-08-06 12:57:14 - INFO - Iniciando descarga de datos DustIQ desde ClickHouse...
2025-08-06 12:57:14 - INFO - Conectando a Clickhouse...
2025-08-06 12:57:15 - INFO - Conexión a Clickhouse establecida
2025-08-06 12:57:15 - INFO - Consultando datos DustIQ desde ClickHouse...
2025-08-06 12:57:15 - INFO - Ejecutando consulta: 
        SELECT 
            Stamptime,
            Attribute,
            Measure
        FROM PSDA...
2025-08-06 12:57:26 - INFO - Datos obtenidos: 1182090 registros
2025-08-06 12:57:26 - INFO - Procesando datos...
2025-08-06 12:57:33 - INFO - Pivotando datos de long format a wide format...
2025-08-06 12:57:33 - INFO - Manejando duplicados agrupando por promedio...
2025-08-06 12:57:34 - INFO - Rango de fechas en los datos:
2025-08-06 12:57:34 - INFO - Fecha más antigua: 2024-07-01 00:00:00+00:00
2025-08-06 12:57:34 - INFO - Fecha más reciente: 2025-08-06 00:00:00+00:00
2025-08-06 12:57:34 - INFO - Guardando datos en: /home/nicole/SR/SOILING/datos/raw_dustiq_data.

True

In [19]:
# 🚀 EJECUCIÓN: DESCARGA SOILING KIT DESDE CLICKHOUSE
# ============================================================================

logger.info("\n" + "="*80)
logger.info("🌪️ DESCARGA SOILING KIT DESDE CLICKHOUSE")
logger.info("="*80)

logger.info(f"�� Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info("🚀 Descarga desde ClickHouse - PSDA.soilingkit")

# Ejecutar descarga del Soiling Kit desde ClickHouse
success = download_soiling_kit_clickhouse(START_DATE, END_DATE, OUTPUT_DIR)

if success:
    # Verificar archivo generado
    output_file = os.path.join(OUTPUT_DIR, 'soiling_kit_raw_data.csv')
    if os.path.exists(output_file):
        file_size_mb = os.path.getsize(output_file) / (1024*1024)
        total_lines = sum(1 for line in open(output_file)) - 1
        
        logger.info("\n" + "="*60)
        logger.info("🎉 ¡DESCARGA SOILING KIT DESDE CLICKHOUSE COMPLETADA!")
        logger.info(f"📂 Archivo: soiling_kit_raw_data.csv")
        logger.info(f"�� Tamaño: {file_size_mb:.2f} MB")
        logger.info(f"�� Total de registros: {total_lines:,}")
        logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
        logger.info("��️ Datos incluyen: Isc(e), Isc(p), Te(C), Tp(C)")
        logger.info("�� Período: Julio 2024 - Julio 2025")
        logger.info("="*60)
        
        # Mostrar muestra de los datos
        try:
            df_sample = pd.read_csv(output_file, nrows=5)
            logger.info("\n📋 Muestra de los datos:")
            logger.info(f"   Columnas: {list(df_sample.columns)}")
            if 'Isc(e)' in df_sample.columns:
                logger.info(f"   Isc(e) rango: {df_sample['Isc(e)'].min():.3f} - {df_sample['Isc(e)'].max():.3f}")
            if 'Isc(p)' in df_sample.columns:
                logger.info(f"   Isc(p) rango: {df_sample['Isc(p)'].min():.3f} - {df_sample['Isc(p)'].max():.3f}")
        except Exception as e:
            logger.warning(f"⚠️ Error al mostrar muestra: {e}")
    else:
        logger.warning("⚠️ Archivo no encontrado después de la descarga")
else:
    logger.error("❌ Error en la descarga del Soiling Kit desde ClickHouse")

logger.info("\n�� DESCARGA SOILING KIT DESDE CLICKHOUSE COMPLETADA")

2025-08-06 12:48:23 - INFO - 
2025-08-06 12:48:23 - INFO - 🌪️ DESCARGA SOILING KIT DESDE CLICKHOUSE
2025-08-06 12:48:23 - INFO - ================================================================================
2025-08-06 12:48:23 - INFO - �� Rango de fechas: 2024-07-01 a 2025-12-31
2025-08-06 12:48:23 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-08-06 12:48:23 - INFO - 🚀 Descarga desde ClickHouse - PSDA.soilingkit
2025-08-06 12:48:23 - INFO - Iniciando descarga de datos del Soiling Kit desde ClickHouse...
2025-08-06 12:48:23 - INFO - Conectando a Clickhouse...
2025-08-06 12:48:24 - INFO - Conexión a Clickhouse establecida
2025-08-06 12:48:24 - INFO - Consultando datos del Soiling Kit desde ClickHouse...
2025-08-06 12:48:24 - INFO - Ejecutando consulta: 
        SELECT 
            Stamptime,
            Attribute,
            Measure
        FROM PSDA...
2025-08-06 12:48:31 - INFO - Datos obtenidos: 1075004 registros
2025-08-06 12:48:31 - INFO - Procesando datos.

In [ ]:
# 🔋 DESCARGA ESPECÍFICA: SOLO IV600
# ============================================================================

logger.info("\n" + "="*80)
logger.info("🔋 DESCARGA ESPECÍFICA: SOLO DATOS IV600")
logger.info("="*80)

logger.info(f"�� Rango de fechas: {START_DATE.strftime('%Y-%m-%d')} a {END_DATE.strftime('%Y-%m-%d')}")
logger.info(f"📁 Directorio de salida: {OUTPUT_DIR}")
logger.info("🚀 Descarga desde ClickHouse - ref_data.iv_curves_trazador_manual")

# Ejecutar descarga específica de IV600
success = download_iv600(START_DATE, END_DATE, OUTPUT_DIR)

if success:
    # Verificar archivo generado
    output_file = os.path.join(OUTPUT_DIR, 'raw_iv600_data.csv')
    if os.path.exists(output_file):
        file_size_mb = os.path.getsize(output_file) / (1024*1024)
        total_lines = sum(1 for line in open(output_file)) - 1
        
        logger.info("\n" + "="*60)
        logger.info("🎉 ¡DESCARGA IV600 COMPLETADA!")
        logger.info(f"�� Archivo: raw_iv600_data.csv")
        logger.info(f"�� Tamaño: {file_size_mb:.2f} MB")
        logger.info(f"�� Total de registros: {total_lines:,}")
        logger.info(f"🗂️ Ubicación: {OUTPUT_DIR}")
        logger.info("🔋 Datos incluyen: timestamp, module, pmp, isc, voc, imp, vmp")
        logger.info("📊 Curvas IV procesadas desde ClickHouse")
        logger.info("="*60)
        
        # Mostrar muestra de los datos
        try:
            df_sample = pd.read_csv(output_file, nrows=5)
            logger.info("\n📋 Muestra de los datos:")
            logger.info(f"   Columnas: {list(df_sample.columns)}")
            if 'pmp' in df_sample.columns:
                logger.info(f"   pmp rango: {df_sample['pmp'].min():.3f} - {df_sample['pmp'].max():.3f}")
            if 'isc' in df_sample.columns:
                logger.info(f"   isc rango: {df_sample['isc'].min():.3f} - {df_sample['isc'].max():.3f}")
            if 'voc' in df_sample.columns:
                logger.info(f"   voc rango: {df_sample['voc'].min():.3f} - {df_sample['voc'].max():.3f}")
        except Exception as e:
            logger.warning(f"⚠️ Error al mostrar muestra: {e}")
    else:
        logger.warning("⚠️ Archivo no encontrado después de la descarga")
else:
    logger.error("❌ Error en la descarga de IV600")

logger.info("\n🏁 DESCARGA IV600 COMPLETADA")

2025-07-29 12:58:10 - INFO - 
2025-07-29 12:58:10 - INFO - 🔋 DESCARGA ESPECÍFICA: SOLO DATOS IV600
2025-07-29 12:58:11 - INFO - ================================================================================
2025-07-29 12:58:11 - INFO - �� Rango de fechas: 2024-07-01 a 2025-12-31
2025-07-29 12:58:11 - INFO - 📁 Directorio de salida: /home/nicole/SR/SOILING/datos
2025-07-29 12:58:11 - INFO - 🚀 Descarga desde ClickHouse - ref_data.iv_curves_trazador_manual
2025-07-29 12:58:11 - INFO - Iniciando descarga de datos IV600...
2025-07-29 12:58:11 - INFO - Conectando a Clickhouse...
2025-07-29 12:58:13 - INFO - Conexión a Clickhouse establecida
2025-07-29 12:58:13 - INFO - Consultando datos IV600...
2025-07-29 12:58:16 - INFO - Datos obtenidos: 2406 registros
2025-07-29 12:58:16 - INFO - Procesando datos...
2025-07-29 12:58:16 - INFO - Creando DataFrame...
2025-07-29 12:58:18 - INFO - Rango de fechas en los datos:
2025-07-29 12:58:18 - INFO - Fecha más antigua: 2024-09-24 12:16:00+00:00
2025-07